## VERSION 2


In [ ]:
from openai import OpenAI
import ast

# Set up OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"  # Replace with your actual key
)

# STEP 1 – Generate Task
def generate_task(goal):
    prompt = f"""
The user is learning: "{goal}".
Generate a short but meaningful persuasive or reflective writing task (3–5 sentences)
that helps assess:
- vocabulary use
- curiosity
- confidence
- compression
- emotional tone

Return just the task instructions, not explanations.
"""
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are a helpful AI tutor."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# STEP 2 – Score User Response
def score_response(goal, task, user_response):
    prompt = f"""
The user is learning: "{goal}"
They were asked to complete this writing task: "{task}"
They wrote the following:

{user_response}

Please rate them between 0.0 and 1.0 on:
1. Vocabulary Expansion
2. Curiosity
3. Confidence
4. Output Compression
5. Emotional Consistency

Return only a Python dictionary like:
{{"vocabulary_expansion_count": ..., "curiosity_level": ..., "confidence_level": ..., "output_compression_ratio": ..., "emotional_consistency_score": ...}}
"""
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are an AI evaluator. Return only a Python dictionary."},
            {"role": "user", "content": prompt}
        ]
    )
    return ast.literal_eval(response.choices[0].message.content)

# STEP 3 – Generate Feedback
def give_feedback(goal, scores):
    prompt = f"""
The user is learning: "{goal}".
Here are their scores:
- Vocabulary Expansion Count: {scores['vocabulary_expansion_count']}
- Curiosity Level: {scores['curiosity_level']}
- Confidence Level: {scores['confidence_level']}
- Output Compression Ratio: {scores['output_compression_ratio']}
- Emotional Consistency Score: {scores['emotional_consistency_score']}

Give:
1. A natural explanation of their learning state
2. Tailored guidance to help them improve in these areas
3. Specific next-step exercises
"""
    response = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are an encouraging but honest AI learning coach."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# ========== ✅ RUN HERE ==========
# Step 0 – Enter Your Learning Goal
goal = input("What are you trying to learn today? ")

# Step 1 – Generate a Personalized Task
task = generate_task(goal)
print("\n📝 TASK FOR YOU TO COMPLETE:\n", task)

# Step 2 – Write Your Answer
user_response = input("\n✍️ Paste your answer here:\n")

# Step 3 – LLM Scores Your Work
scores = score_response(goal, task, user_response)
print("\n📊 SCORES:", scores)

# Step 4 – Get LLM-Powered Feedback
feedback = give_feedback(goal, scores)
print("\n💡 FEEDBACK:\n", feedback)


In [ ]:
from openai import OpenAI
import json
import random
import time

# Setup client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",  # Replace with your real key
)

# Core traits to evaluate
traits = [
    "vocabulary_expansion_count",
    "curiosity_level",
    "confidence_level",
    "output_compression_ratio",
    "emotional_consistency_score"
]

# Task generator (indirect prompt style)
def generate_learning_task(goal):
    writing_scenarios = [
        "Recall the last time you felt stuck while writing. What were you writing about, and what was holding you back?",
        "Describe a time you had to convince someone to agree with you. How did you try to persuade them?",
        "Write 3-5 sentences about a moment where you changed your opinion on something important. What led to the change?",
        "Share a time when you had to write about something you didn’t care about. How did you try to make it work?",
        "Reflect on the most rewarding piece of writing you've ever done. Why did it feel that way?"
    ]
    return random.choice(writing_scenarios)

# Trait scoring function
def score_response(goal, task, response_text):
    prompt = f"""You are a learning assessment AI. A student wants to get better at: "{goal}"

They were given this task:
"{task}"

Here is their response:
"{response_text}"

Now rate their performance on the following traits from 0.0 to 1.0:

1. vocabulary_expansion_count – Variety and strength of vocabulary used.
2. curiosity_level – Depth of exploration and insight into the topic.
3. confidence_level – Strength and assertiveness of tone.
4. output_compression_ratio – Clarity and conciseness.
5. emotional_consistency_score – Stability and effectiveness of emotional tone.

Return only a JSON object with the trait names as keys and scores as float values. Do NOT include any other explanation, markdown, or code formatting.
"""

    completion = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528-qwen3-8b:free",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4,
        max_tokens=400
    )

    try:
        # Attempt to extract and parse JSON directly
        raw = completion.choices[0].message.content.strip()
        return json.loads(raw)
    except json.JSONDecodeError:
        print("⚠️ Could not decode response:", raw)
        return None

# Example use
goal = input("What are you trying to learn today? ")
task = generate_learning_task(goal)
print("\n📝 TASK FOR YOU TO COMPLETE:")
print(task)

# Let the user enter response manually
user_response = input("\n✍️ Paste your answer here:\n")

# Get scores
scores = score_response(goal, task, user_response)

# Display scores if successful
if scores:
    print("\n--- METRIC SCORES ---")
    for trait, value in scores.items():
        print(f"{trait}: {value:.2f}")
else:
    print("❌ Scoring failed. Please check formatting or try again.")
